In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import os
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import PIL.Image as Image
import numpy as np

In [14]:
input_folder = ".\\combined\\"
komp_weight = ".\\weights\\komp.pth"
birads_weight = ".\\weights\\birads.pth"
excel_save_path = ".\\results.xlsx"
input_size = 832

In [15]:
data_transform = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [16]:
inputs = glob(input_folder + "*.png")
# birads, komp
birads_res = []
komp_res = []
results = []

In [18]:
# 10 dk
def birads_predict(inputs):
    birads_model = torch.load(birads_weight)
    birads_model = birads_model.cuda()
    birads_model.eval()
    for input in tqdm(inputs):
        input = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])(Image.open(input))
        input =  Variable(input.cuda())
        output = birads_model(input.unsqueeze(0))
        _, preds = torch.max(output.data, 1)
        if(preds.item() == 0):
            birads_res.append("BI-RADS0")
        elif(preds.item() == 1):
            birads_res.append("BI-RADS1-2")
        elif(preds.item() == 2):
            birads_res.append("BI-RADS4-5")

In [19]:
# 10 dk
def komp_predict(inputs):
    komp_model = torch.load(komp_weight)
    komp_model = komp_model.cuda()
    komp_model.eval()
    for input in tqdm(inputs):
        input = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])(Image.open(input))
        input =  Variable(input.cuda())
        output = komp_model(input.unsqueeze(0))
        _, preds = torch.max(output.data, 1)
        if(preds.item() == 0):
            komp_res.append("A")
        elif(preds.item() == 1):
            komp_res.append("B")
        elif(preds.item() == 2):
            komp_res.append("C")
        elif(preds.item() == 3):
            komp_res.append("D")

In [20]:
birads_predict(inputs)

100%|██████████| 10/10 [00:03<00:00,  2.57it/s]


In [21]:
komp_predict(inputs)

100%|██████████| 10/10 [00:03<00:00,  2.63it/s]


In [23]:
if(len(birads_res) != len(komp_res)):
    print("Fatal Error")

In [24]:
for i in range(len(birads_res)):
    results.append({"HASTANO": inputs[i].split("\\")[-1].split(".")[0], "BIRADS KATEGORİSİ": birads_res[i], "MEME KOMPOZİSYONU": komp_res[i], "KADRAN BİLGİSİ (SAĞ)": None, "KADRAN BİLGİSİ (SOL)": None})

In [25]:
res_data_frame = pd.DataFrame.from_records(results)
print(res_data_frame)

     HASTANO BIRADS KATEGORİSİ MEME KOMPOZİSYONU KADRAN BİLGİSİ (SAĞ)  \
0  822702935        BI-RADS1-2                 A                 None   
1  822702942        BI-RADS1-2                 A                 None   
2  822702950        BI-RADS1-2                 A                 None   
3  822702964        BI-RADS1-2                 B                 None   
4  825890613        BI-RADS4-5                 C                 None   
5  825890617        BI-RADS1-2                 D                 None   
6  836163882        BI-RADS1-2                 D                 None   
7  836187148        BI-RADS1-2                 C                 None   
8  836194547        BI-RADS4-5                 B                 None   
9  845282653        BI-RADS1-2                 C                 None   

  KADRAN BİLGİSİ (SOL)  
0                 None  
1                 None  
2                 None  
3                 None  
4                 None  
5                 None  
6                 Non

In [12]:
pd.io.formats.excel.ExcelFormatter.header_style = None
res_data_frame.to_excel(excel_save_path, index=False)